# SpecMetrics - Processing from AWS

In [ ]:
import json, os, sys
from tqdm import tqdm
import pandas as pd

ROOT_DIR = os.environ['HOME'] + '/Dev/spec_metrics_dashboard'
sys.path.append(ROOT_DIR + '/src/lib')

import connector_s3, process

In [ ]:
run_keys = process.fetch_run_keys(connector_s3)
runs_df = process.build_runs_df(run_keys)
runs_df.index = runs_df.date

branch_names = process.branch_names(runs_df=runs_df)
print(len(branch_names), "branches")

In [ ]:
run_key = "raw/jobteaser-jobteaser/20160928181433126-develop-79b5bf12"
run_data = process.fetch_run_data(connector_s3, run_key)
examples_df = process.build_run_examples_df(run_data)

In [ ]:
level_0_paths = sorted(set(examples_df.path_0))
for level_0_path in level_0_paths:
    level_1_paths = examples_df[examples_df.path_0 == level_0_path].path_1

In [ ]:
examples_df.groupby(["path_0", "path_1", "path_2", "path_3", "path_4"]).sum()

## Identifying recurring failing examples solved by rerun

Count of unique failed examples: 
- 56 over last 50 runs
- 75 over last 100 runs
- 147 over last 200 runs

In [ ]:
all_failed_examples_df = pd.DataFrame()
develop_run_keys = list(runs_df[runs_df.branch == "develop"].run_key)
for key in tqdm(list(reversed(develop_run_keys))[0:200]):
    data = process.fetch_run_data(connector_s3, key)
    examples_df = process.build_run_examples_df(data)
    if len(examples_df) == 0:
        # we may have runs with no examples, in this case the df is empty
        continue
    failed_examples_df = examples_df[examples_df.status == "failed"]
    all_failed_examples_df = all_failed_examples_df.append(failed_examples_df, ignore_index=True)

print("Count of found unique failed examples over N last develop runs:", len(all_failed_examples_df.drop_duplicates("description")))

In [ ]:
most_failed_example_description = all_failed_examples_df.groupby("description") \
    .count().sort_values(by="path_0", ascending=False).index[0]
most_failed_example_description

In [ ]:
all_failed_examples_df.groupby("description") \
    .count()

Trying to find the examples that were run before the most failed example ran. Must isolate them in the case the most failed example failed. We may also isolate the tests that run before when it didn't fail, to have white and blacklists.

In [ ]:
# Find all examples that failed before the most failed example,
# add them to "all_before_most_failed_examples_df" and get
# insights from this.
#
# NB: this is the union, not the intersection!

LIMIT = 50
all_before_most_failed_examples_df = pd.DataFrame()
count_of_runs_with_most_failed_example_failed = 0

develop_run_keys = list(runs_df[runs_df.branch == "develop"].run_key)
for key in tqdm(list(reversed(develop_run_keys))[0:LIMIT]):
    data = process.fetch_run_data(connector_s3, key)
    examples_df = process.build_run_examples_df(data)

    if len(examples_df) == 0:
        # no examples in this run
        continue
    
    most_failed_example = examples_df[examples_df.description == most_failed_example_description]
    if len(most_failed_example) == 0:
        # most failed example not present in this run
        continue
    
    if most_failed_example.status.iloc[0] == "passed":
        # the most failed example did not fail in this run, ignoring it
        continue
        
    count_of_runs_with_most_failed_example_failed += 1

    examples_df.index = examples_df.finished_at
    most_failed_example_finished_at = most_failed_example.finished_at.iloc[0]
    before_most_failed_examples_df = examples_df[examples_df.finished_at < most_failed_example_finished_at]
    
    all_before_most_failed_examples_df = all_before_most_failed_examples_df.append(
        before_most_failed_examples_df, ignore_index=True)

In [ ]:
failure_source_examples_df = all_before_most_failed_examples_df[["description", "path_0"]].groupby("description").count()
failure_source_examples_df.columns = ["count"]
failure_source_examples_df["presence"] = failure_source_examples_df["count"] / count_of_runs_with_most_failed_example_failed
failure_source_examples_df[failure_source_examples_df.presence > 0.9].sort_values(by="count", ascending=False)

## Intersection of examples present before a given failing example

In [ ]:
# Find all examples that failed before the most failed example,
# build the intersection of their descriptions and get insights
# from this.
#
# NB: this is the intersection!
#
# TODO: use pandas dataframe intersection instead!

LIMIT = 50
before_most_failed_example_descriptions_intersection = []

develop_run_keys = list(runs_df[runs_df.branch == "develop"].run_key)
for key in tqdm(list(reversed(develop_run_keys))[0:LIMIT]):
    data = process.fetch_run_data(connector_s3, key)
    examples_df = process.build_run_examples_df(data)

    if len(examples_df) == 0:
        # no examples in this run
        continue
    
    most_failed_example = examples_df[examples_df.description == most_failed_example_description]
    if len(most_failed_example) == 0:
        # most failed example not present in this run
        continue
    
    if most_failed_example.status.iloc[0] == "passed":
        # the most failed example did not fail in this run, ignoring it
        continue
        
    examples_df.index = examples_df.finished_at
    most_failed_example_finished_at = most_failed_example.finished_at.iloc[0]
    before_most_failed_examples_df = examples_df[examples_df.finished_at < most_failed_example_finished_at]
    before_most_failed_example_descriptions = list(set(before_most_failed_examples_df.description))

    if len(before_most_failed_example_descriptions_intersection) == 0:
        # empty, add first descriptions before intersecting
        before_most_failed_example_descriptions_intersection += before_most_failed_example_descriptions
    else:
        # intersecting
        before_most_failed_example_descriptions_intersection = list(
            set(before_most_failed_example_descriptions_intersection) & \
            set(before_most_failed_example_descriptions)
        )

In [ ]:
data = process.fetch_run_data(connector_s3, develop_run_keys[-2])
examples_df = process.build_run_examples_df(data)
examples_df[examples_df.description == 'StudentNews#send_news_notifications should have a job queued with ["StudentMailer", "news_recap", "student_profile_id"]']

## Chart

In [ ]:
import highcharts
from IPython.display import display, HTML

%run setup_highcharts.py
load_highcharts()

In [ ]:
load_highcharts_modules()

In [ ]:
branch_runs_df = runs_df[runs_df.branch == "develop"]
run_key = branch_runs_df.to_dict(orient="rows")[-22]["run_key"]
run_data = process.fetch_run_data(connector_s3, run_key)
run_examples_df = process.build_run_examples_df(run_data)

chart_df = run_examples_df[["path_0", "path_1", "run_time"]] \
    .groupby(['path_0', 'path_1']) \
    .sum()[['run_time']]
    
chart_js = highcharts.pie_drilldown(
    chart_df,
    serie_name='Run time',
    title="Run time",
    percentage=False,
    unit="s"
)

display(HTML("<div id='chart'><script>" + chart_js +"</script>"))